In [ ]:
from data_preprocess import preprocess_h5ad
import scanpy as sc
import anndata

In [2]:
adata1=preprocess_h5ad("/volume1/home/pxie/data/PBMC.h5ad")
adata2=preprocess_h5ad("/volume1/home/pxie/data/Cortex.h5ad")



In [10]:
import pandas as pd

# 首先执行基本合并（不要使用 join 参数，它可能导致问题）
combined = sc.concat([adata1, adata2], label='batch')

# 手动创建新的 var 数据框
# 确定最终的基因集（应该等于 combined.var_names）
all_genes = combined.var_names

# 创建新的 var 数据框
new_var = pd.DataFrame(index=all_genes)

# 添加 feature_name 列
new_var['feature_name'] = pd.Series(dtype='object', index=all_genes)
new_var['ensembl_id'] = pd.Series(dtype='object', index=all_genes)

# 为每个数据集覆盖相应的基因
for gene in adata1.var_names:
    if gene in all_genes:
        idx1 = adata1.var_names.get_loc(gene)
        new_var.loc[gene, 'feature_name'] = adata1.var['feature_name'].iloc[idx1]
        new_var.loc[gene, 'ensembl_id'] = adata1.var['ensembl_id'].iloc[idx1]

for gene in adata2.var_names:
    if gene in all_genes and gene not in adata1.var_names:
        idx2 = adata2.var_names.get_loc(gene)
        new_var.loc[gene, 'feature_name'] = adata2.var['feature_name'].iloc[idx2]
        new_var.loc[gene, 'ensembl_id'] = adata2.var['ensembl_id'].iloc[idx2]

# 将新的 var 数据框分配给合并后的对象
combined.var = new_var

# 验证结果
print("Combined shape after fix:", combined.shape)
print("Combined var shape after fix:", combined.var.shape)
print("Combined var columns after fix:", combined.var.columns.tolist())

Combined shape after fix: (87455, 21432)
Combined var shape after fix: (21432, 2)
Combined var columns after fix: ['feature_name', 'ensembl_id']


In [11]:
combined

AnnData object with n_obs × n_vars = 87455 × 21432
    obs: 'nCount_RNA', 'nFeature_RNA', 'sample', 'donor_id', 'sex_ontology_term_id', 'percent.mt', 'development_stage_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'suspension_type', 'is_primary_data', 'tissue_type', 'tissue_ontology_term_id', 'organism_ontology_term_id', 'disease_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'n_genes', 'batch'
    var: 'feature_name', 'ensembl_id'
    obsm: 'X_umap'

In [12]:
combined.write_h5ad('/volume1/home/pxie/data/combined_data.h5ad')